In [1]:
import mlflow
import mlflow.pyfunc
import mlflow.spark
import mlflow.mleap
from mlflow.tracking.client import MlflowClient

#twitter
import tweepy as tw
import pandas as pd
import re

nombreModelo = "Clasificador NaiveBayes Twitter"
stageModelo = "Production"


In [2]:
client = MlflowClient()
model_version = client.get_latest_versions(nombreModelo, stages=[stageModelo])[0].version
model_uri = "models:/{model_name}/{model_stage}".format(model_name=nombreModelo, model_stage=stageModelo)
model = mlflow.spark.load_model(model_uri)


In [3]:
consumer_key= ''
consumer_secret= ''
access_token= '16644668-'
access_token_secret= ''

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Define the search term and the date_since date as variables
search_words = "amlo"
date_since = "2020-04-01"

new_search = search_words + " -filter:retweets"

# Collect tweets
tweets = tw.Cursor(api.search,
              q=new_search,
              lang="es",
              tweet_mode="extended",
              since=date_since).items(1000)
 
users_locs = [[tweet.user.screen_name,re.sub(r"http\S+", "", tweet.full_text)] for tweet in tweets]

tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user','texto'])

datosNuevoTwitter = spark.createDataFrame(tweet_text)
#tweet_text.to_csv('tweets.csv',encoding='utf-8')
                    
display (datosNuevoTwitter)

user,texto
britishilango,@hdemauleon Y ustedes que que pensaban que AMLO no sabia ler ni escrebir
candycortc,UFFF AGRADECIDA POR TENER A AMLO DE PRESI😫😫🥵🥵🥵
loredo60,"Y así, el #NARCOGÜELITO se convirtió en aquello que por tantos años criticó, a veces con razón, las más por rencor político puro. Así los tiranos wannabe..."
emilianodanisr,te odio amlo!!!
potrogdl,@proceso #TeamCacas no escucha dictador comúnista desea más pobreza e ignorancia cuando chingados van a entender chairos #amlo #LOPEZnosEstasQuebrando
BotTita2,"@CruzMisburck @CitlaHM @lopezobrador_ El que les dijo mascotas fue AMLO y no, yo no quiero convencerte de nada, no sé de dónde sacas esa idea."
CISCOXXIII,Saben la miseria que gana un profe de escuela particular? Y ahora hay que ponerle de la bolsa para las clases en linea. Se pasaron de lanzas @SEP_mx @emoctezumab haber si ya mejor cierran la boca. Total AMLO acabara con el pais.
OMNIANOTICIAS,"#ESTADO Propone Corral a AMLO otorgar estímulos a empresas en IMSS, Infonavit y Afore @Javier_Corral @lopezobrador_ @GobiernoEdoChih"
EjeCentral,"Las medidas implementadas por el gobierno de @lopezobrador_ son ""tímidas"" por lo que es poco probable que mitiguen el impacto económico de la crisis de #COVIDー19, afirma Eurasia Group (@EurasiaGroup), consultora de riesgo político"
Knigthrep,"@HistoriaPedro @Enea_Salgado Lo siento Pedrito, no pude conseguir uno con la cara de AMLO, se nota que te hace falta uno..."


In [4]:
resultados = model.transform(datosNuevoTwitter)
display(resultados.select("texto","probability","prediction"))

texto,probability,prediction
@hdemauleon Y ustedes que que pensaban que AMLO no sabia ler ni escrebir,"List(1, 3, List(), List(0.22651034996210317, 0.34389352326509337, 0.4295961267728034))",2.0
UFFF AGRADECIDA POR TENER A AMLO DE PRESI😫😫🥵🥵🥵,"List(1, 3, List(), List(0.4112359550561799, 0.3707865168539324, 0.21797752808988763))",0.0
"Y así, el #NARCOGÜELITO se convirtió en aquello que por tantos años criticó, a veces con razón, las más por rencor político puro. Así los tiranos wannabe...","List(1, 3, List(), List(0.5620730664091264, 0.22169583146533586, 0.21623110212553767))",0.0
te odio amlo!!!,"List(1, 3, List(), List(0.4112359550561799, 0.3707865168539324, 0.21797752808988763))",0.0
@proceso #TeamCacas no escucha dictador comúnista desea más pobreza e ignorancia cuando chingados van a entender chairos #amlo #LOPEZnosEstasQuebrando,"List(1, 3, List(), List(0.45897756107323834, 0.2508591232906627, 0.290163315636099))",0.0
"@CruzMisburck @CitlaHM @lopezobrador_ El que les dijo mascotas fue AMLO y no, yo no quiero convencerte de nada, no sé de dónde sacas esa idea.","List(1, 3, List(), List(0.21168821572046598, 0.5502869833983414, 0.2380248008811927))",1.0
Saben la miseria que gana un profe de escuela particular? Y ahora hay que ponerle de la bolsa para las clases en linea. Se pasaron de lanzas @SEP_mx @emoctezumab haber si ya mejor cierran la boca. Total AMLO acabara con el pais.,"List(1, 3, List(), List(0.17511773108410103, 0.12335366833162736, 0.7015286005842717))",2.0
"#ESTADO Propone Corral a AMLO otorgar estímulos a empresas en IMSS, Infonavit y Afore @Javier_Corral @lopezobrador_ @GobiernoEdoChih","List(1, 3, List(), List(0.5529605332997425, 0.3226883587409645, 0.12435110795929309))",0.0
"Las medidas implementadas por el gobierno de @lopezobrador_ son ""tímidas"" por lo que es poco probable que mitiguen el impacto económico de la crisis de #COVIDー19, afirma Eurasia Group (@EurasiaGroup), consultora de riesgo político","List(1, 3, List(), List(0.3902063142590043, 0.3423059655320144, 0.26748772020898126))",0.0
"@HistoriaPedro @Enea_Salgado Lo siento Pedrito, no pude conseguir uno con la cara de AMLO, se nota que te hace falta uno...","List(1, 3, List(), List(0.19754952538089035, 0.5939123400090712, 0.20853813461003842))",1.0
